# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import seaborn as sns
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from scipy import stats as st
import numpy as np
from numpy.random import RandomState

In [3]:
df_1 = pd.read_csv('geo_data_0.csv')
df_2 = pd.read_csv('geo_data_1.csv')
df_3 = pd.read_csv('geo_data_2.csv')

In [4]:
# Функция ознакомления с данными
def table_of_contents(df):
    display(df.head())
    print(df.shape)
    print(df.describe())

In [5]:
print('Первый регион:')
table_of_contents(df_1)
print('................', '\n')

print('Второй регион:')
table_of_contents(df_2)
print('................', '\n')

print('Третий регион:')
table_of_contents(df_3)

Первый регион:


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


(100000, 5)
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
................ 

Второй регион:


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


(100000, 5)
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%         1.153055      -4.813172       2.011479      57.085625
75%         8.621015      -1.332816       3.999904     107.813044
max        29.421755      18.734063       5.019721     137.945408
................ 

Третий регион:


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


(100000, 5)
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335       0.000000
25%        -1.162288      -1.174820       0.130359      59.450441
50%         0.009424      -0.009482       2.484236      94.925613
75%         1.158535       1.163678       4.858794     130.595027
max         7.238262       7.844801      16.739402     190.029838


## Обучение и проверка модели

**Целевой признак - объём запасов в скважине (тыс. баррелей). Остальные столбцы - признаки точек**

In [6]:
def ml(df):
    features = df.drop(['product', 'id'], axis = 1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = 0.25, random_state = 12345)
    # Объявление модели
    model = LinearRegression()

    # Обучене модели на тренировочной выборке
    model.fit(features_train, target_train)

    # Предсказания модели на валидационной выборке
    predictions = model.predict(features_valid)

    # Вычисление MSE и RMSE
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse ** 0.5

    # Вычисление среднего запаса предсказанного сырья
    mean_prod = predictions.mean()
    
    #print('Средний запас предсказанного сырья: ', mean_prod)
    #print('RMSE: ', rmse)
    return(mean_prod, rmse, predictions, target_valid)

**Вывод результатов:**

In [7]:
print('Первый регион:', '\n')
mean_prod, rmse, predictions_1, target_valid_1 = ml(df_1)
print('Средний запас предсказанного сырья: ', mean_prod)
print('RMSE: ', rmse)
print('................', '\n')


print('Второй регион:', '\n')
mean_prod, rmse, predictions_2, target_valid_2 = ml(df_2)
print('Средний запас предсказанного сырья: ', mean_prod)
print('RMSE: ', rmse)
print('................', '\n')


print('Третий регион:', '\n')
mean_prod, rmse, predictions_3, target_valid_3 = ml(df_3)
print('Средний запас предсказанного сырья: ', mean_prod)
print('RMSE: ', rmse)
print('................', '\n')

Первый регион: 

Средний запас предсказанного сырья:  92.59256778438035
RMSE:  37.5794217150813
................ 

Второй регион: 

Средний запас предсказанного сырья:  68.728546895446
RMSE:  0.893099286775617
................ 

Третий регион: 

Средний запас предсказанного сырья:  94.96504596800489
RMSE:  40.02970873393434
................ 



**Регион 1:**
- Второе место по запасу сырья и по показателю средней ошибки

**Регион 2:**
- Самый низкий запас сырья, но самый низкий показатель ошибки

**Регион 3:**
- Самый высокий запас сырья, но при этом самый высокий показатель ошибки

## Подготовка к расчёту прибыли

In [9]:
DEV_BUDGET = 10**10  # Бюджет на разработку всех скважин в регионе (руб)

AMOUNT_REG = 200   # Количество скважин в регионе (по условию - выбирают 200 точек)

INCOME_1PROD = 450000  # Доход от 1 тыс.баррелей (руб)

In [10]:
print('Необходимый объем сырья для региона:')
VOL = DEV_BUDGET / INCOME_1PROD
print(VOL, 'тыс.баррелей', '\n')

print('Необходимый объем сырья для одной скважины в среднем:')
print(VOL/AMOUNT_REG, 'тыс.баррелей', '\n')

INCOME_ONE = DEV_BUDGET / AMOUNT_REG
print('Необходимый средний доход с одной скважины:')
print(INCOME_ONE)

Необходимый объем сырья для региона:
22222.222222222223 тыс.баррелей 

Необходимый объем сырья для одной скважины в среднем:
111.11111111111111 тыс.баррелей 

Необходимый средний доход с одной скважины:
50000000.0


**Если сравнивать результат, то окажется, что необходимый объем для безубыточной разработки выше, чем средний запас в каждом регионе**

In [11]:
# Функция перевода predictions в series и сброса индексов в target
def ser(predictions, target):
    predictions = pd.Series(predictions)
    target = target.reset_index(drop = True)
    return predictions, target
predictions_1, target_valid_1 = ser(predictions_1, target_valid_1)
predictions_2, target_valid_2 = ser(predictions_2, target_valid_2)
predictions_3, target_valid_3 = ser(predictions_3, target_valid_3)

**Функция расчета прибыли**

In [13]:
def profit(target, predictions):
    
    # Расчет запсов для целевых значений количества сырья по индексам лучших предсказаний модели
    pr_sorted = predictions.sort_values(ascending=False)
    selected = target[pr_sorted.index][:200]
    
    # Просуммируем результат
    summ = selected.sum() 
    
    # Вычтем бюджет из суммы, чтобы сразу получить чистую прибыль
    return round((summ * INCOME_1PROD) - DEV_BUDGET,2)

## Расчёт прибыли и рисков 

In [16]:
def calculation(target, predictions):
    state = np.random.RandomState(12345)
    
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = 500, replace = True, random_state = state)
        pr_subsample = predictions[target_subsample.index]  
        values.append(profit(target_subsample, pr_subsample))

    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()

    print("Процент риска", values.apply(lambda x: x < 0).sum()/len(values)*100,"%")
    print("Средняя прибыль:", mean, 'руб')
    print("2.5%-квантиль:", lower,)
    print("97.5%-квантиль:", upper)

In [17]:
print('Первый регион:')
calculation(target_valid_1, predictions_1)
print('.................', '\n')

print('Второй регион:')
calculation(target_valid_2, predictions_2)
print('.................', '\n')

print('Третий регион:')
calculation(target_valid_3, predictions_3)

Первый регион:
Процент риска 6.0 %
Средняя прибыль: 425938526.91045994 руб
2.5%-квантиль: -102090094.84225
97.5%-квантиль: 947976353.3625
................. 

Второй регион:
Процент риска 1.0 %
Средняя прибыль: 515222773.44336 руб
2.5%-квантиль: 68873225.366
97.5%-квантиль: 931547591.2524999
................. 

Третий регион:
Процент риска 6.4 %
Средняя прибыль: 435008362.78263 руб
2.5%-квантиль: -128880547.3315
97.5%-квантиль: 969706954.183


**Рассмотрим полученные характеристики**

**Регион 1:**
- Процент риска: 6%
- 95%-ый доверительный интервал: [-102 млн 090 тыс; 947 млн 976 тыс]
- Средняя прибыль: 425 млн 938 тыс рублей

**Регион 2:**
- Процент риска: 1.0%
- 95%-ый доверительный интервал: [68 млн 873 тыс; 931 млн 547 тыс]
- Средняя прибыль: 515 млн 222 тыс рублей

**Регион 3:**
- Процент риска: 6.4%
- 95%-ый доверительный интервал: [-128 млн 880 тыс; 969 млн 706 тыс]
- Средняя прибыль: 435 млн 8 тыс рублей

**Согласно условию, регион должен принести 10 млрд рублей.**
- Вероятность убытка в первом регионе больше 2.5%, т.к. 2.5% квантилю соответствует значение -102 млн. Этот регион не подходит под условие  
- Второй регион подходит под условие и имеет наименьший процент риска. Также второй регион имеет большую среднюю прибыль  
- Третий регион имеет процент риска выше 6.4% и 2.5% квантиль соответствует отрицательной прибыли, а, значит, под условие не подходит. Данный регион больше не рассматривается

**Рекомендация для разработки: регион 2.  
Высокий потенциал обусловлен низким проентом риска, а также высокой прибылью**